## Work
請結合前面的知識與程式碼，比較不同的 regularization 的組合對訓練的結果與影響：如 dropout, regularizers, batch-normalization 等

In [1]:
import keras
import itertools
from keras.models import Sequential
from keras.layers import Dense,Dropout,BatchNormalization
from keras.regularizers import l1
import matplotlib.pyplot as plt
from keras import optimizers
%matplotlib inline
# Disable GPU
#os.environ["CUDA_VISIBLE_DEVICES"] = ""

Using TensorFlow backend.


In [2]:
train, test = keras.datasets.cifar10.load_data()

In [3]:
## 資料前處理
def preproc_x(x, flatten=True):
    x = x / 255.
    if flatten:
        x = x.reshape((len(x), -1))
    return x

def preproc_y(y, num_classes=10):
    if y.shape[-1] == 1:
        y = keras.utils.to_categorical(y, num_classes)
    return y    

In [4]:
x_train, y_train = train
x_test, y_test = test

# Preproc the inputs
x_train = preproc_x(x_train)
x_test = preproc_x(x_test)

# Preprc the outputs
y_train = preproc_y(y_train)
y_test = preproc_y(y_test)

In [5]:
# Hint 1 : 在 Dense 層中增加 Regularizers
# Hint 2 : 增加 Dropout 層並設定 dropout ratio 
# Hint 3 : 增加 Batch-normalization 層
def build_mlp(input_shape, output_units=10, num_neurons=[512, 256, 128],Regu_rate=1e-4,drp_ratio=0.2):
    
    model = Sequential()
    model.add(Dense(units = input_shape[1] ,input_dim = input_shape[1], kernel_initializer = 'normal', activation = 'relu',name = 'input'))
    
    for i, n_units in enumerate(num_neurons):
        model.add(Dense(units=n_units, kernel_initializer='normal', activation='relu',name="hidden_layer"+str(i+1), kernel_regularizer=l1(Regu_rate)))  
        model.add(Dropout(drp_ratio))  
        model.add(BatchNormalization())         
    
    model.add(Dense(units=output_units,kernel_initializer='normal', activation='softmax',name='output'))

    return model

In [6]:
def plot_model_score(results):
    
    NUM_COLORS = len(results.keys())
    cm = plt.get_cmap('gist_rainbow')
    color_bar = [cm(1.*i/NUM_COLORS) for i in range(NUM_COLORS)]
    
    plt.figure(figsize=(10,8))
    for i, cond in enumerate(results.keys()):
        plt.plot(range(len(results[cond]['train-loss'])),results[cond]['train-loss'], '-', label='train-'+cond, color=color_bar[i])
        plt.plot(range(len(results[cond]['valid-loss'])),results[cond]['valid-loss'], '--', label='valid-'+cond, color=color_bar[i])
    plt.title("Loss")
    
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()


    plt.figure(figsize=(10,8))
    for i, cond in enumerate(results.keys()):
        plt.plot(range(len(results[cond]['train-acc'])),results[cond]['train-acc'], '-', label='train-'+cond, color=color_bar[i])
        plt.plot(range(len(results[cond]['valid-acc'])),results[cond]['valid-acc'], '--', label='valid-'+cond, color=color_bar[i])
    plt.title("Accuracy")
    
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.show()

In [7]:
## 超參數設定
"""
Set your hyper-parameters
"""
LEARNING_RATE = 1e-3
EPOCHS = 50
BATCH_SIZE = [256,1024]
MOMENTUM = 0.95
L1_EXP = [1e-2,  1e-8]
Dropout_EXP = [0.2,0.6]

In [ ]:
results = {}

for regu_rate,drop_rate,batch_size in itertools.product(L1_EXP,Dropout_EXP,BATCH_SIZE):
    keras.backend.clear_session() # 把舊的 Graph 清掉
    model = build_mlp(input_shape=x_train.shape,Regu_rate=regu_rate,drp_ratio=drop_rate)
    model.summary()
    optimizer = keras.optimizers.SGD(lr=LEARNING_RATE, nesterov=True, momentum=MOMENTUM)
    model.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer=optimizer)

    model.fit(x_train, y_train, 
              epochs=EPOCHS, 
              batch_size=batch_size, 
              validation_data=(x_test, y_test), 
              shuffle=True)

    # Collect results
    train_loss = model.history.history["loss"]
    valid_loss = model.history.history["val_loss"]
    train_acc = model.history.history["acc"]
    valid_acc = model.history.history["val_acc"]
    
    exp_name_tag = f'L1:{str(regu_rate)},DR: {str(drop_rate)},BS:{str(batch_size)}'
    results[exp_name_tag] = {'train-loss': train_loss,
                             'valid-loss': valid_loss,
                             'train-acc': train_acc,
                             'valid-acc': valid_acc}

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 3072)              9440256   
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 512)               2048      
_________________________________________________________________
hidden_layer2 (Dense)        (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 256)               1024      
__________

Epoch 47/50
50000/50000 [==============================] - 88s 2ms/step - loss: 2.0285 - acc: 0.4161 - val_loss: 2.0455 - val_acc: 0.4155
Epoch 48/50
50000/50000 [==============================] - 87s 2ms/step - loss: 2.0240 - acc: 0.4160 - val_loss: 2.0345 - val_acc: 0.4132
Epoch 49/50
50000/50000 [==============================] - 87s 2ms/step - loss: 2.0236 - acc: 0.4161 - val_loss: 2.0435 - val_acc: 0.4268
Epoch 50/50
50000/50000 [==============================] - 87s 2ms/step - loss: 2.0211 - acc: 0.4173 - val_loss: 2.0306 - val_acc: 0.4119
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (Dense)                (None, 3072)              9440256   
_________________________________________________________________
hidden_layer1 (Dense)        (None, 512)               1573376   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)            

Epoch 43/50
50000/50000 [==============================] - 69s 1ms/step - loss: 2.1835 - acc: 0.3910 - val_loss: 2.1914 - val_acc: 0.3925
Epoch 44/50
50000/50000 [==============================] - 69s 1ms/step - loss: 2.1769 - acc: 0.3924 - val_loss: 2.2055 - val_acc: 0.3811
Epoch 45/50
50000/50000 [==============================] - 69s 1ms/step - loss: 2.1815 - acc: 0.3936 - val_loss: 2.1809 - val_acc: 0.3954
Epoch 46/50
50000/50000 [==============================] - 69s 1ms/step - loss: 2.1784 - acc: 0.3948 - val_loss: 2.3072 - val_acc: 0.3599
Epoch 47/50
50000/50000 [==============================] - 70s 1ms/step - loss: 2.1695 - acc: 0.4018 - val_loss: 2.2167 - val_acc: 0.3769
Epoch 48/50
50000/50000 [==============================] - 70s 1ms/step - loss: 2.1660 - acc: 0.3982 - val_loss: 2.2031 - val_acc: 0.3830
Epoch 49/50
50000/50000 [==============================] - 69s 1ms/step - loss: 2.1582 - acc: 0.4014 - val_loss: 2.1840 - val_acc: 0.4055
Epoch 50/50
50000/50000 [=========

Epoch 39/50
50000/50000 [==============================] - 85s 2ms/step - loss: 2.3893 - acc: 0.2125 - val_loss: 2.3016 - val_acc: 0.2618
Epoch 40/50
50000/50000 [==============================] - 85s 2ms/step - loss: 2.3897 - acc: 0.2149 - val_loss: 2.2841 - val_acc: 0.2500
Epoch 41/50
50000/50000 [==============================] - 85s 2ms/step - loss: 2.3873 - acc: 0.2141 - val_loss: 2.3265 - val_acc: 0.2550
Epoch 42/50
50000/50000 [==============================] - 86s 2ms/step - loss: 2.3921 - acc: 0.2171 - val_loss: 2.3298 - val_acc: 0.2408
Epoch 43/50
50000/50000 [==============================] - 86s 2ms/step - loss: 2.3946 - acc: 0.2149 - val_loss: 2.3045 - val_acc: 0.2651
Epoch 44/50
50000/50000 [==============================] - 85s 2ms/step - loss: 2.3853 - acc: 0.2160 - val_loss: 2.2915 - val_acc: 0.2625
Epoch 45/50
50000/50000 [==============================] - 86s 2ms/step - loss: 2.3921 - acc: 0.2154 - val_loss: 2.3188 - val_acc: 0.2243
Epoch 46/50
50000/50000 [=========

Epoch 35/50
50000/50000 [==============================] - 67s 1ms/step - loss: 2.4747 - acc: 0.2132 - val_loss: 2.4147 - val_acc: 0.2423
Epoch 36/50
50000/50000 [==============================] - 67s 1ms/step - loss: 2.4255 - acc: 0.2205 - val_loss: 2.3553 - val_acc: 0.2513
Epoch 37/50
50000/50000 [==============================] - 67s 1ms/step - loss: 2.4253 - acc: 0.2171 - val_loss: 2.4016 - val_acc: 0.2352
Epoch 38/50
50000/50000 [==============================] - 67s 1ms/step - loss: 2.4338 - acc: 0.2102 - val_loss: 2.4200 - val_acc: 0.1984
Epoch 39/50
50000/50000 [==============================] - 67s 1ms/step - loss: 2.4230 - acc: 0.2162 - val_loss: 2.4725 - val_acc: 0.2044
Epoch 40/50
50000/50000 [==============================] - 67s 1ms/step - loss: 2.4180 - acc: 0.2143 - val_loss: 2.3984 - val_acc: 0.2341
Epoch 41/50
50000/50000 [==============================] - 67s 1ms/step - loss: 2.4215 - acc: 0.2188 - val_loss: 2.4657 - val_acc: 0.2049
Epoch 42/50
50000/50000 [=========

50000/50000 [==============================] - 84s 2ms/step - loss: 1.2238 - acc: 0.5595 - val_loss: 1.3540 - val_acc: 0.5225
Epoch 32/50
50000/50000 [==============================] - 84s 2ms/step - loss: 1.2096 - acc: 0.5646 - val_loss: 1.3472 - val_acc: 0.5260
Epoch 33/50
50000/50000 [==============================] - 85s 2ms/step - loss: 1.1994 - acc: 0.5677 - val_loss: 1.3488 - val_acc: 0.5191
Epoch 34/50
50000/50000 [==============================] - 85s 2ms/step - loss: 1.1865 - acc: 0.5732 - val_loss: 1.3335 - val_acc: 0.5271
Epoch 35/50
50000/50000 [==============================] - 84s 2ms/step - loss: 1.1745 - acc: 0.5765 - val_loss: 1.3666 - val_acc: 0.5175
Epoch 36/50
50000/50000 [==============================] - 84s 2ms/step - loss: 1.1612 - acc: 0.5815 - val_loss: 1.3484 - val_acc: 0.5335
Epoch 37/50
50000/50000 [==============================] - 85s 2ms/step - loss: 1.1543 - acc: 0.5864 - val_loss: 1.3312 - val_acc: 0.5278
Epoch 38/50
50000/50000 [=====================

In [ ]:
plot_model_score(results)
"""
Plot results
"""